```
    __
   (__      /_      _        /
  (__  (_/ (   (-  (  /) (- . 
                     /
```

## MongoDB API Tests

Tests for updating `trackdb` and interation with class `euterpe.track.track`.  
Author: `@hughshuwang`  

In [1]:
from __future__ import print_function

import sys
sys.path.append('../') 
# adding euterpe 

import os
import numpy as np
import pickle 
# pickle.dumps(x, protocol=2)

import librosa as rs
import librosa.feature as ft
import librosa.display as dp
import librosa.segment as sg

import pymongo
from pymongo import MongoClient

from itertools import izip
from bson.binary import Binary

from euterpe.utils import *
from euterpe.track import track

import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline

DB_PATH = '../data-raw/tracks/'
FILE_TYPE = 'mp3'

# $: sudo mongod # required 
client = MongoClient()
client.list_database_names()

trackdb = client['trackdb']
meta = trackdb['meta']

# TODO: drop duplicates in mongodb
# ref: https://stackoverflow.com/questions/6367589/saving-numpy-array-in-mongodb

### Directory Info Extraction
Generate a list and a dict using `./tracks`

In [2]:
tracklist = np.array(gen_tracklist(DB_PATH))
trackdict = gen_trackdict(DB_PATH) # can change field

stamp_default = {'duration' : 5, 'offset' : 0.0}
# duration should be None, 5 for speed now
map(lambda x: x.update(stamp_default), trackdict)

dictin = trackdict[0]
dictin

{'album': 'Frank',
 'artist': 'Amy Winehouse',
 'duration': 5,
 'name': 'What Is It About Men',
 'offset': 0.0}

### Packing `track` objects and API tests
Generate dict outputs using `track` objects and update `trackdb.info`

In [3]:
outputs = []
for dictin in trackdict:
    t = track(**dictin)
    t.load(DB_PATH, FILE_TYPE) # fill (y, sr) 
    outputs.append(t.gen_dict()) # autotrans ndarray 

# outputs[0] # test case, with y transferred to list

In [4]:
for output in outputs:
    key_info = ['album', 'artist', 'name', 'offset', 'duration']
    # basic and stamp info for identifying each section/track
    
    query = {k: output[k] for k in key_info}
    meta.replace_one(query, output) 
    # if new info gened
    # insert_one if for the first time

In [5]:
list(meta.find({'artist' : 'Frank Ocean'}, 
               {'album' : 1, 'artist' : 1,
                'name' : 1, 'offset' : 1, 
                'duration' : 1}))

[{u'_id': ObjectId('5c0f3f9f907b080521e8de61'),
  u'album': u'Endless',
  u'artist': u'Frank Ocean',
  u'duration': 5.0,
  u'name': u'Rushes',
  u'offset': 0.0},
 {u'_id': ObjectId('5c0f3f9f907b080521e8de62'),
  u'album': u'Channel Orange',
  u'artist': u'Frank Ocean',
  u'duration': 5.0,
  u'name': u'Pyramids',
  u'offset': 0.0},
 {u'_id': ObjectId('5c0f3f9f907b080521e8de63'),
  u'album': u'Channel Orange',
  u'artist': u'Frank Ocean',
  u'duration': 5.0,
  u'name': u'Pink Matter',
  u'offset': 0.0}]

In [6]:
dictout = list(meta.find({'name' : 'Pink Matter'}))[0]
sample_y = np.array(dictout['y'])
sample_sr = dictout['sr']

In [7]:
ipd.Audio(sample_y, rate = sample_sr)
# first 5 seconds of PIIIINK MATTER

In [8]:
client.close()